In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from tf_fourier_features import FourierFeatureProjection
from tf_fourier_features import FourierFeatureMLP

# Load data
X = np.load('Datasets/kryptonite-45-X.npy')
y = np.load('Datasets/kryptonite-45-y.npy')

# Preprocess X to ensure values are either 0 or 1
X = np.round(X)  # Round to nearest integer, resulting in 0s or 1s

# Verify preprocessing
print("Processed X data (sample):", X[:5])  # Display a sample to verify 0s and 1s only

# Split into train, validation, and test sets (70-15-15 split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

# Define a neural network with Fourier feature projection and dense layers for binary classification

ip = tf.keras.layers.Input(shape=[X.shape[1]])
x = FourierFeatureProjection(gaussian_projection=10, gaussian_scale=10)(ip)
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

model = tf.keras.Model(inputs=ip, outputs=output)

# Compile the model with binary cross-entropy loss and accuracy metric
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=128,
    verbose=1,
    callbacks=[early_stopping]
)

# Plot the loss curves for training and validation
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on the test set for loss and accuracy
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Processed X data (sample): [[ 1.  1.  1.  0. -0.  0.  0.  0.  1. -0.  1.  1.  1.  1. -0.  0. -0.  1.
   0.  0.  1.  0.  0. -0.  1. -0.  1.  1.  1.  1.  0.  0.  1.  0.  1.  1.
   1.  0.  0.  1. -0.  1.  1.  1.  1.]
 [ 1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  0.  1.  0. -0.  1.
   1.  0.  0.  0.  1.  1.  1.  1.  1.  0.  1. -0.  1.  0. -0.  1.  1. -0.
   1.  0.  1. -0. -0.  0.  1. -0.  1.]
 [-0.  1.  1.  1.  1.  0.  0.  1.  0.  0.  1.  1. -0.  1.  1.  1.  0. -0.
   1.  1.  1. -0.  1.  0.  1.  1.  1. -0.  1.  1. -0. -0. -0.  1. -0. -0.
  -0. -0.  1.  1.  1.  1.  1. -0. -0.]
 [ 1.  0.  1.  1. -0. -0.  1. -0. -0.  1.  0.  0.  1.  0.  1.  0.  0.  0.
   1.  1.  0.  1. -0.  1. -0.  1.  1.  1. -0.  1.  0. -0.  1. -0.  0.  0.
   0.  1.  0.  1.  1.  0. -0.  1.  0.]
 [ 1.  0.  0. -0. -0.  1.  1. -0.  0.  0. -0.  1.  0.  1.  1.  1.  1.  0.
  -0.  1.  1.  1.  0.  1.  0. -0. -0.  0.  0.  1.  1.  0.  1.  0.  1.  0.
   1.  0. -0.  1. -0.  1.  1.  0.  1.]]
Training set: (63000, 45) (63000,)
Va